# Visualizing The Total Number of Outstanding Cases
> Visualizing Total Number of Outstanding Cases by Geographic Area.

- comments: true
- author: Adrian Turcato
- categories: [growth, compare, interactive, d3]
- hide: false
- image: images/covid-outstanding-cases.png
- permalink: /outstanding_cases/

## Outstanding Cases by Geography

Number of outstanding cases, i.e. number of individuals who are still currently ill. The color of the country reflects the reported case fatality rate (CFR), these rates do not reflect the true fatality rate of the virus but are heavily affected by the amount of testing which is performed in the country.


> Tip: Click the buttons to toggle between geographies, or to change the scale. 

In [1]:
#hide
from IPython.display import HTML, Javascript, display
from string import Template
import numpy as np
import pandas as pd
import json

In [2]:
#hide
display(Javascript("require.config({paths: {d3: 'https://d3js.org/d3.v2.min.js'}});"))

<IPython.core.display.Javascript object>

In [3]:
#hide
def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_19-covid-{name}.csv')
    df = pd.read_csv(url)
    return df

def flatten(df):
    array = []
    d = df.to_dict()
    for v in d.items():
        day = v[0]
        for c in v[1].items():
            country = c[0]
            value = c[1]
            array.append({"key":country,"value":value,"date":day})
    return array

def combine(cx,dx,rx):
    all = []
    for i, v in enumerate(cx):
        v['confirmed'] = v['value']
        v['deaths'] = dx[i]['value']
        v['recovered'] = rx[i]['value']
        v['value'] = v['value'] - dx[i]['value'] - rx[i]['value']
        if v['value'] < 0:
            print("-1 error:", v['value'], v['confirmed'], dx[i]['value'], rx[i]['value'])
            v['value'] = 0
        fr = 0
        if rx[i]['value'] + dx[i]['value'] + v['value'] > 0:
            fr = dx[i]['value'] / (rx[i]['value'] + dx[i]['value'] + v['value'])
        v['fatality'] = fr
        all.append(v)
    return all;

In [4]:
#hide
def extractGlobal(df):
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])
    df = df.groupby('Country/Region').sum()
    return df

abbreviations=[{"name":"Alabama","abbreviation":"AL"},{"name":"Alaska","abbreviation":"AK"},{"name":"American Samoa","abbreviation":"AS"},{"name":"Arizona","abbreviation":"AZ"},{"name":"Arkansas","abbreviation":"AR"},{"name":"California","abbreviation":"CA"},{"name":"Colorado","abbreviation":"CO"},{"name":"Connecticut","abbreviation":"CT"},{"name":"Delaware","abbreviation":"DE"},{"name":"District of Columbia","abbreviation":"D.C."},{"name":"Federated States Of Micronesia","abbreviation":"FM"},{"name":"Florida","abbreviation":"FL"},{"name":"Georgia","abbreviation":"GA"},{"name":"Guam","abbreviation":"GU"},{"name":"Hawaii","abbreviation":"HI"},{"name":"Idaho","abbreviation":"ID"},{"name":"Illinois","abbreviation":"IL"},{"name":"Indiana","abbreviation":"IN"},{"name":"Iowa","abbreviation":"IA"},{"name":"Kansas","abbreviation":"KS"},{"name":"Kentucky","abbreviation":"KY"},{"name":"Louisiana","abbreviation":"LA"},{"name":"Maine","abbreviation":"ME"},{"name":"Marshall Islands","abbreviation":"MH"},{"name":"Maryland","abbreviation":"MD"},{"name":"Massachusetts","abbreviation":"MA"},{"name":"Michigan","abbreviation":"MI"},{"name":"Minnesota","abbreviation":"MN"},{"name":"Mississippi","abbreviation":"MS"},{"name":"Missouri","abbreviation":"MO"},{"name":"Montana","abbreviation":"MT"},{"name":"Nebraska","abbreviation":"NE"},{"name":"Nevada","abbreviation":"NV"},{"name":"New Hampshire","abbreviation":"NH"},{"name":"New Jersey","abbreviation":"NJ"},{"name":"New Mexico","abbreviation":"NM"},{"name":"New York","abbreviation":"NY"},{"name":"North Carolina","abbreviation":"NC"},{"name":"North Dakota","abbreviation":"ND"},{"name":"Northern Mariana Islands","abbreviation":"MP"},{"name":"Ohio","abbreviation":"OH"},{"name":"Oklahoma","abbreviation":"OK"},{"name":"Oregon","abbreviation":"OR"},{"name":"Palau","abbreviation":"PW"},{"name":"Pennsylvania","abbreviation":"PA"},{"name":"Puerto Rico","abbreviation":"PR"},{"name":"Rhode Island","abbreviation":"RI"},{"name":"South Carolina","abbreviation":"SC"},{"name":"South Dakota","abbreviation":"SD"},{"name":"Tennessee","abbreviation":"TN"},{"name":"Texas","abbreviation":"TX"},{"name":"Utah","abbreviation":"UT"},{"name":"Vermont","abbreviation":"VT"},{"name":"Virgin Islands","abbreviation":"VI"},{"name":"Virginia","abbreviation":"VA"},{"name":"Washington","abbreviation":"WA"},{"name":"West Virginia","abbreviation":"WV"},{"name":"Wisconsin","abbreviation":"WI"},{"name":"Wyoming","abbreviation":"WY"}]

def replaceStates(y):
    x = y.split(",")
    if len(x) > 1:
        for i, e in enumerate(abbreviations):
            if e['abbreviation'].strip() == x[1].strip():
                return e['name']
    if y == "United States Virgin Islands":
        return "Virgin Islands"
    return y

def extractUSA(df):
    df = df[df['Country/Region'] == "US"]
    df = df.drop(columns=['Country/Region', 'Lat', 'Long'])
    df = df[~df['Province/State'].isin(["US"])]
    df['Province/State'] = df['Province/State'].apply(lambda x: replaceStates(x))
    df = df.groupby('Province/State').sum()
    dates = df.columns[:32].values
    df = df.drop(columns=dates)
    return df

def extractCanada(df):
    df = df[df['Country/Region'] == "Canada"]
    df = df.drop(columns=['Country/Region', 'Lat', 'Long'])
    df = df.groupby('Province/State').sum()
    dates = df.columns[:32].values
    df = df.drop(columns=dates)
    return df

eu = ['Germany','Finland','Italy','Spain','Belgium','Switzerland','Austria','Greece','Norway','Romania','Estonia','San Marino','Belarus','Iceland','Lithuania','Ireland','Luxembourg','Monaco','Portugal','Andorra','Latvia','Ukraine' 'Hungary','Liechtenstein','Poland','Bosnia and Herzegovina','Slovenia','Serbia','Slovakia','Bulgaria','Albania','Holy See','France','Denmark','Czechia','Moldova','United Kingdom','Kosovo','Netherlands','Montenegro']

def extractEurope(df):
    df = df[df['Country/Region'].isin(eu)]
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])
    df = df.groupby('Country/Region').sum()
    datesEU = df.columns[:32].values
    df = df.drop(columns=datesEU)
    return df

In [5]:
#hide
confirmedRaw = get_frame('Confirmed')
deathsRaw = get_frame('Deaths')
recoveredRaw = get_frame('Recovered')

In [6]:
#hide
extractGlobal(confirmedRaw)

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,11,16,21,22,22,22,24,24,40,40.0
Albania,0,0,0,0,0,0,0,0,0,0,...,38,42,51,55,59,64,70,76,89,89.0
Algeria,0,0,0,0,0,0,0,0,0,0,...,37,48,54,60,74,87,90,139,201,201.0
Andorra,0,0,0,0,0,0,0,0,0,0,...,1,1,2,39,39,53,75,88,113,113.0
Angola,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,2,2,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uzbekistan,0,0,0,0,0,0,0,0,0,0,...,0,1,6,10,15,23,33,43,43,43.0
Venezuela,0,0,0,0,0,0,0,0,0,0,...,2,10,17,33,36,42,42,70,70,70.0
Vietnam,0,2,2,2,2,2,2,2,2,2,...,53,56,61,66,75,85,91,94,113,113.0


In [7]:
#hide
def prepData(extract,varname):
    confirmed = extract(confirmedRaw)
    c = flatten(confirmed)
    recovered = extract(recoveredRaw)
    r = flatten(recovered)
    deaths = extract(deathsRaw)
    d = flatten(deaths)
    all = combine(c,d,r)
    json_string = json.dumps(all)
    
    pre = "var " + varname + " ="
    post = ";" #"; get" + varname + " = function (){ return " + varname + "};})();"
    script = pre + json_string + post
    
    return script

In [8]:
#hide
getGlobalDataJson = prepData(extractGlobal,"globalData")
getUsaDataJson = prepData(extractUSA,"usaData")
getCanadaDataJson = prepData(extractCanada,"canadaData")
getEuropeDataJson = prepData(extractEurope,"europeData")

-1 error: -1.0 0.0 1.0 0.0
-1 error: -1.0 0.0 1.0 0.0


In [9]:
#hide
html_temp = Template('''
    <script src="https://d3js.org/d3.v2.min.js"></script>
    <style scoped>
        $css_text 
    </style>
    <div id="streamgraph">
		<div class="buttons">
			<button onclick="toggleGlobal()">Global</button>
			<button onclick="toggleEU()">Europe</button>
			<button onclick="toggleUSA()">USA</button>
			<button onclick="toggleCanada()">Canada</button>
		</div>
		<button onclick="toggleScale()" id="toggle">Toggle Scale</button>
		<div class="chart">
		</div>
	</div>
    <script>
        $getUSA
        $getEU
        $getGlobal
        $getCanada
        
        $d3_script
        
    </script>
''')

In [10]:
#hide
css_text = '''
.chart {
  font: 10px sans-serif;
  background: #fff;
  height: 575px;
}

#tooltip {
  font: 10px sans-serif;
}

.axis path, .axis line {
  fill: none;
  stroke: #000;
  stroke-width: 1px;
  shape-rendering: crispEdges;
}

.buttons {
  font: 11px sans-serif;
  position: relative;
  left: 50px;
  top: 10px;
}

#toggle {
  font: 11px sans-serif;
  position: relative;
  left: 660px;
  top: 10px;
}'''

In [11]:
#hide
d3_script = '''
    console.log("d3",d3)
    var margin, width, height;
    var svg, tooltip, area, blank;
    var y, x, yAxis, xAxis;

    var data, layers, logScale;
    var usaLayers, globalLayers, euLayers;
    var dataClass, prevClass;

    margin = {top: 20, right: 60, bottom: 100, left: 30};
    width = 750 - margin.left - margin.right;
    height = 550 - margin.top - margin.bottom;

    svg = d3.select(".chart").append("svg")
        .attr("width", width + margin.left + margin.right)
        .attr("height", height + margin.top + margin.bottom)
        .append("g")
        .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

    var cs = d3.scale.linear()
        .domain([0,0.5,1])
        .interpolate(d3.interpolateRgb)
        .range([d3.rgb('#cc0000'),d3.rgb(249, 247, 174),d3.rgb(0, 104, 55)]);

    var format = d3.time.format("%m/%d/%y");
    function formatDate (d) {
        d.date = format.parse(d.date);
        d.value = +d.value;
    } 

    // axis
    x = d3.time.scale()
        .range([0, width])
        .clamp(true);

    xAxis = d3.svg.axis()
        .orient("bottom")
        .ticks(d3.time.weeks);
        // .tickFormat(d3.time.format("%b %d"));

    svg.append("g")
        .attr("class", "x axis")
        .attr("transform", "translate(0," + height + ")");		

    yAxis = d3.svg.axis()
        .orient("right") 
        .tickFormat(d3.format(","));

    svg.append("g")
        .attr("class", "y axis")
        .attr("transform", "translate(" + width + ", 0)");

    // clipping paths
    svg.append("defs")
        .append("clipPath")
        .attr("id","clip")
        .append("rect")
        .attr("width",width-3)
        .attr("height",height-10)
        .attr("x",0)
        .attr("y",0);

    svg.append("text")
        .attr("text-anchor", "middle") 
        .attr("transform", "translate("+ (width + margin.right*0.8) +","+(height/2)+")rotate(90)")  
        .text("Current Outstanding Cases");

    svg.append("text")
        .attr("text-anchor", "middle")  
        .attr("transform", "translate("+ (width/2) +","+(height + 40)+")")  
        .text("Date");

    area = d3.svg.area()
        .interpolate("cardinal")
        .x(d => x(d.date));

    blank = d3.svg.area()
        .interpolate("cardinal")
        .x(d => x(d.date));

    usaData.forEach(formatDate);
    usaLayers = prepareData(usaData);

    globalData.forEach(formatDate);
    globalLayers = prepareData(globalData);

    europeData.forEach(formatDate);
    euLayers = prepareData(europeData);

    canadaData.forEach(formatDate);
    canadaLayers = prepareData(canadaData);

    dataClass = "globalData";
    prevClass = "usaData";

    data = globalData;
    layers = globalLayers;
    logScale = true;

    drawScale();
    drawLegend();
    draw();

    toggleScale = function (){
        logScale = !logScale;
        drawScale();
    }
    toggleUSA = function(){
        prevClass = dataClass;
        dataClass = "usaData";

        data = usaData;
        layers = usaLayers;
        drawScale();
        draw();
    }
    toggleGlobal = function (){
        prevClass = dataClass;
        dataClass = "globalData";

        data = globalData;
        layers = globalLayers;
        drawScale(); 
        draw();
    }
    toggleEU = function(){
        prevClass = dataClass;
        dataClass = "europeData";

        data = europeData;
        layers = euLayers;
        drawScale();
        draw();
    }
    toggleCanada = function(){
        prevClass = dataClass;
        dataClass = "canadaData";

        data = canadaData;
        layers = canadaLayers;
        drawScale();
        draw();
    }
    function drawScale(){
        area.y0(d => {return y(d.y0 + 1)})
            .y1(d => {return y(d.y0 + d.y + 1)});
        blank.y0(d => height)
            .y1(d => height);

        var yExt = d3.extent(data, d => {return d.y0 + d.y});
        var logMin = 1000;
        if(dataClass == "usaData") logMin = 300;
        if(dataClass == "canadaData") logMin = 10;
        y = d3.scale.log()
            .range([height-10, 0])
            .domain([logMin,yExt[1]])
            .clamp(true);
        if(!logScale){
            y = d3.scale.linear()
                .range([height-10, 0])
                .domain(yExt)
                .clamp(true);
        }
        yAxis.scale(y);

        x.domain(d3.extent(data, d => d.date));
        xAxis.scale(x);

        svg.selectAll(".y")
            .transition("axis")
            .duration(1000)
            .call(yAxis);

        svg.selectAll(".x")
            .transition("axis")
            .duration(1000)
            .call(xAxis);
            // .selectAll("text")  
            // .style("text-anchor", "end")
            // .attr("dx", "-.8em")
            // .attr("dy", ".15em")
            // .attr("transform", "rotate(-65)" );

        svg.selectAll(".layer")
            .transition("axis")
            .duration(1000)
            .attr("d", d => area(d.values))	
    }
    function draw(){
        var leaving = svg.selectAll(`.${prevClass}`);
        leaving.transition("load")
            .duration(1000)
            .attr("d", d => blank(d.values))
            .remove();

        var countries = svg.selectAll(`.${dataClass}`)
            .data(layers);

        countries.enter()
            .append("path")
            .attr("d", d => blank(d.values))
            .on("mousemove", d => updateTooltip(d))
            .on("mouseout", function(d, i) {
                tooltip.style("visibility", "hidden");
            })
            .transition("load")
            .duration(1000)
            .attr("d", d => blank(d.values))
            .attr("class", `layer ${dataClass}`)
            .attr("d", d => area(d.values))
            .attr("fill", d => {return `url(#${d.gradient})`})
            .attr("clip-path","url(#clip)")
            .attr("opacity", 1);
    }
    function drawLegend(){
        var defs = svg.append("defs");

        var gradient = defs.append("linearGradient")
           .attr("id", "legend-grad")
           .attr("x1", "0%")
           .attr("x2", "100%")
           .attr("y1", "0%")
           .attr("y2", "0%");

        for(var i = 0; i <= 10; i++){
            gradient.append("stop")
                .attr("offset", `${i*10}%`)
                .attr("stop-color", cs(1-i/10)) // d3.interpolateRdYlGn(1-i/10))
                .attr("stop-opacity", 1);
        }

        var legend = svg.append("g")
            .attr("id","legend")
            .attr("transform", `translate(${(width/2)-150},${height+65})`)
        legend.append("rect")
            .attr("width",300)
            .attr("height",15)
            .attr("fill", "url(#legend-grad)");
        legend.append("text")
            .attr("text-anchor", "end") 
            .attr("transform", "translate("+ -10 +","+10+")")  
            .text("Case Fatality Rate (CFR)");

        var lScale = d3.scale.linear()
            .range([0, 300])
            .domain([0.0,0.09]);

        var lAxis = d3.svg.axis()
            .orient("bottom") 
            .scale(lScale)
            .tickSize(18,0)
            .tickFormat(d3.format(".1%"));

        var l2 = svg.append("g")
            .attr("class", "legend-axis axis")
            .attr("transform", `translate(${(width/2)-150},${height+65})`)
            .call(lAxis);

        l2.selectAll("path")
            .remove();
    }
    function prepareData(data){
        console.log("fn: prepareData")
        //manipulate data
        var nest = d3.nest()
            .key(function(d) { return d.key; });
        var alphabetic = nest.entries(data);
        var l = alphabetic.length;

        //order data
        for(var i = 0; i < l; i++){
            var vals = alphabetic[i].values;
            var lv = vals.length;
            var current = vals[lv-1];
            alphabetic[i].fatality = current.fatality;
            alphabetic[i].current = current.value;
            alphabetic[i].index = i
        }
        alphabetic.sort((a,b) => (a.current - b.current));

        var stack = d3.layout.stack()
          .offset("zero")
          .order(() => alphabetic.map(e => e.index))
          .values(d => d.values)
          .x(d => d.date)
          .y(d => d.value);

        //add color gradients
        for(var i = 0; i < l; i++){
            var name = alphabetic[i].key.replace(/\s/g, '')
            var id = `gradient-${name}`;
            alphabetic[i].gradient = id;

            var defs = svg.append("defs");

            var gradient = defs.append("linearGradient")
               .attr("id", id)
               .attr("x1", "0%")
               .attr("x2", "100%")
               .attr("y1", "0%")
               .attr("y2", "0%");

            var values = alphabetic[i].values;
            var lv = alphabetic[i].values.length;

            for(var j = 0; j < lv; j++){
                var offset = `${Math.round(j*100/(lv-1))}%`;
                var fr = values[j].fatality/0.09;
                var col = cs(1-fr); //d3.interpolateRdYlGn(1-fr);

                gradient.append("stop")
                    .attr("offset", offset)
                    .attr("stop-color", col)
                    .attr("stop-opacity", 1);  
            }
        }

        var layers = stack(nest.entries(data));
        for(var i = 0; i < l; i++){
            var name = layers[i].key.replace(/\s/g, '')
            var id = `gradient-${name}`;
            layers[i].gradient = id;
        }
        return layers;
    }
    function updateTooltip(d){
        current = d3.format(",")(d.values[d.values.length-1].value)
        deaths = d3.format(",")(d.values[d.values.length-1].deaths)
        recoveries = d3.format(",")(d.values[d.values.length-1].recovered)
        fatality = d3.format(".1%")(d.values[d.values.length-1].fatality)
        day = d3.time.format("%m/%d/%y")(d.values[d.values.length-1].date)
        
        var n = d.key;
		if(n == "Korea, South") n = "South Korea";
		if(n == "Taiwan*") n = "Taiwan";

        tooltip = svg.selectAll(".tooltip")
            .data([n,day,`current cases: ${current}`,`deaths: ${deaths}`,`recoveries: ${recoveries}`,`CFR: ${fatality}`]);
        tooltip.style("visibility", "visible")
            .text(d => d);
        tooltip.enter()
            .append("text")
            .attr("class", "tooltip")
            .attr("x","0")
            .attr("y",(d,i) => `${i}em`)
            .attr("font-weight", (d,i) => i == 0 ? "bold" : "normal")
            .attr("opacity",1)
            .text(d => d);
    }
'''

In [12]:
#hide
html_text = html_temp.substitute({
    'css_text':css_text
    ,'d3_script':d3_script
    ,"getUSA":getUsaDataJson
    ,"getEU":getEuropeDataJson
    ,"getGlobal":getGlobalDataJson
    ,"getCanada":getCanadaDataJson
})

In [13]:
#hide_input
HTML(html_text)

This visualization was created by [Adrian Turcato](https://www.linkedin.com/in/adrian-turcato-9a543185/). Relevant sources are listed below: 

1. [2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). 
